In [ ]:
# --- CÀI ĐẶT MÔI TRƯỜNG ---
# Giả định bạn đã clone repo từ GitHub vào môi trường Kaggle
# Lệnh này sẽ cài đặt các dependencies từ file requirements.txt
!pip install -q -e ./Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import random
import json
from tqdm.notebook import tqdm
import os

In [ ]:
# Import các module từ dự án của bạn
# Đảm bảo đường dẫn chính xác sau khi clone repo
import sys
sys.path.append('/kaggle/working/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication')

from dataloader.meta_dataloader import SignatureEpisodeDataset
from models.feature_extractor import ResNetFeatureExtractor
from models.meta_learner import MetricGenerator
from losses.triplet_loss import adaptive_mahalanobis_triplet_loss
from utils.model_evaluation import evaluate_meta_model

print("Cài đặt và import thành công!")

In [ ]:
# Đường dẫn (QUAN TRỌNG: Cần cập nhật trên Kaggle)
# Giả sử bạn đã upload file cedar_meta_split.json lên một dataset trên Kaggle
SPLIT_FILE_PATH = '/kaggle/input/cedar-meta-split/cedar_meta_split.json'

# Cấu hình Meta-Learning
K_SHOT = 5          # Số mẫu chữ ký thật trong support set
N_QUERY_GENUINE = 5 # Số mẫu chữ ký thật trong query set
N_QUERY_FORGERY = 5 # Số mẫu chữ ký giả trong query set

# Cấu hình Mô hình
EMBEDDING_DIM = 512 # Kích thước vector embedding từ ResNet-34

# Cấu hình Huấn luyện
NUM_EPOCHS = 100
LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Seed để đảm bảo kết quả có thể tái lặp
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"Sử dụng thiết bị: {DEVICE}")
print(f"Cấu hình: {K_SHOT}-shot learning")

In [ ]:
# --- KHỞI TẠO CÁC THÀNH PHẦN ---

# 1. Transform ảnh (giữ nguyên từ dự án cũ)
transform = transforms.Compose([
    transforms.Resize((220, 150)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 2. Dataset và DataLoader cho Meta-Training
# Batch size luôn là 1 vì mỗi item trả về là một "episode" hoàn chỉnh
train_dataset = SignatureEpisodeDataset(
    split_file_path=SPLIT_FILE_PATH,
    split_name='meta-train',
    k_shot=K_SHOT,
    n_query_genuine=N_QUERY_GENUINE,
    n_query_forgery=N_QUERY_FORGERY,
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2)

print(f"Đã tạo meta-train dataset với {len(train_dataset)} người dùng (tasks).")

# 3. Khởi tạo Models
# Backbone (Feature Extractor) - Tái sử dụng từ dự án cũ
feature_extractor = ResNetFeatureExtractor(backbone_name='resnet34', output_dim=EMBEDDING_DIM).to(DEVICE)
# Meta-learner mới
metric_generator = MetricGenerator(embedding_dim=EMBEDDING_DIM).to(DEVICE)

# Xử lý đa GPU nếu có
if torch.cuda.device_count() > 1:
    print(f"Sử dụng {torch.cuda.device_count()} GPUs!")
    feature_extractor = nn.DataParallel(feature_extractor)
    metric_generator = nn.DataParallel(metric_generator)

# 4. Khởi tạo Optimizer
# Tối ưu hóa cả hai mạng cùng lúc
optimizer = optim.Adam(
    list(feature_extractor.parameters()) + list(metric_generator.parameters()),
    lr=LEARNING_RATE
)

In [ ]:
# --- BẮT ĐẦU VÒNG LẶP META-TRAINING ---

print("Bắt đầu huấn luyện...")
for epoch in range(NUM_EPOCHS):
    feature_extractor.train()
    metric_generator.train()
    
    total_epoch_loss = 0.0
    
    # Sử dụng tqdm để theo dõi tiến trình
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False)
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        # 1. Lấy dữ liệu episode và chuyển lên GPU
        # .squeeze(0) để loại bỏ chiều batch_size=1
        support_images = batch['support_images'].squeeze(0).to(DEVICE)
        query_images = batch['query_images'].squeeze(0).to(DEVICE)
        query_labels = batch['query_labels'].squeeze(0).to(DEVICE)
        
        # 2. Trích xuất embeddings
        # Nối support và query để xử lý qua mạng một lần cho hiệu quả
        all_images = torch.cat([support_images, query_images], dim=0)
        all_embeddings = feature_extractor(all_images)
        
        support_embeddings = all_embeddings[:K_SHOT]
        query_embeddings = all_embeddings[K_SHOT:]
        
        # 3. Sinh ra metric thích ứng (ma trận W)
        # Nếu dùng DataParallel, cần truy cập module bên trong
        gen_module = metric_generator.module if isinstance(metric_generator, nn.DataParallel) else metric_generator
        W = gen_module(support_embeddings)
        
        # 4. Tạo triplets từ query set để tính loss
        genuine_query_embeddings = query_embeddings[query_labels == 1]
        forgery_query_embeddings = query_embeddings[query_labels == 0]
        
        # Logic tạo triplets đơn giản: Lấy 1 cặp thật và N cặp giả
        if len(genuine_query_embeddings) > 1 and len(forgery_query_embeddings) > 0:
            anchor_feat = genuine_query_embeddings[0]
            positive_feat = genuine_query_embeddings[1]
            
            # Tính loss cho từng mẫu giả
            episode_loss = 0.0
            num_triplets = 0
            for i in range(len(forgery_query_embeddings)):
                negative_feat = forgery_query_embeddings[i]
                
                # Tính loss cho 1 triplet
                loss = adaptive_mahalanobis_triplet_loss(
                    anchor_feat.unsqueeze(0), 
                    positive_feat.unsqueeze(0), 
                    negative_feat.unsqueeze(0), 
                    W
                )
                episode_loss += loss
                num_triplets += 1
            
            if num_triplets > 0:
                avg_episode_loss = episode_loss / num_triplets
                
                # 5. Lan truyền ngược và cập nhật
                avg_episode_loss.backward()
                optimizer.step()
                
                total_epoch_loss += avg_episode_loss.item()
                progress_bar.set_postfix(loss=avg_episode_loss.item())

    avg_epoch_loss = total_epoch_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - Average Loss: {avg_epoch_loss:.4f}")

print("Hoàn thành huấn luyện!")

In [ ]:
# --- ĐÁNH GIÁ TRÊN TẬP META-TEST ---

print("Bắt đầu đánh giá trên meta-test set (người dùng chưa từng thấy)...")

# Chuyển các model về 1 GPU để đánh giá cho đơn giản
fe_eval = feature_extractor.module if isinstance(feature_extractor, nn.DataParallel) else feature_extractor
mg_eval = metric_generator.module if isinstance(metric_generator, nn.DataParallel) else metric_generator

meta_test_accuracy = evaluate_meta_model(
    feature_extractor=fe_eval,
    metric_generator=mg_eval,
    test_split_path=SPLIT_FILE_PATH,
    transform=transform,
    k_shot=K_SHOT,
    device=DEVICE
)

print(f"\n>>>>> Few-Shot Accuracy trên Meta-Test Set: {meta_test_accuracy * 100:.2f}% <<<<<")

In [ ]:
# --- LƯU MODEL ---

# Tạo thư mục lưu
SAVE_DIR = '/kaggle/working/meta_models'
os.makedirs(SAVE_DIR, exist_ok=True)

# Lấy ra state_dict từ model (xử lý DataParallel)
fe_state_dict = feature_extractor.module.state_dict() if isinstance(feature_extractor, nn.DataParallel) else feature_extractor.state_dict()
mg_state_dict = metric_generator.module.state_dict() if isinstance(metric_generator, nn.DataParallel) else metric_generator.state_dict()

# Lưu
torch.save(fe_state_dict, os.path.join(SAVE_DIR, 'feature_extractor.pth'))
torch.save(mg_state_dict, os.path.join(SAVE_DIR, 'metric_generator.pth'))

print(f"Đã lưu các model đã huấn luyện vào thư mục: {SAVE_DIR}")

# Bạn có thể nén thư mục này lại và tải về, hoặc đẩy trực tiếp lên GitHub/Kaggle Models
# !zip -r meta_models.zip /kaggle/working/meta_models